In [1]:
import pandas as pd

## Analyze all log statements 

### Load the sampled log statements

In [2]:
pickle_path = "../data/sampled_logs.csv"
sampled_logs = pd.read_csv(pickle_path)

In [3]:
sampled_logs

,Project,log ID,Log,Log level,Log URL,Code(double click),Log content relation,Log-code relation
0,activemq,19634,"LOG.debug(""{} started XA transaction {}"", this...",debug,https://github.com/apache/activemq/blob/main/a...,private void setXid(Xid xid) throws XAExceptio...,NaN,right after
1,activemq,19295,"LOG.trace(""MQTT-->ActiveMQ: MQTT_MSGID:{} clie...",trace,https://github.com/apache/activemq/blob/main/a...,ActiveMQMessage convertMessage(PUBLISH command...,NaN,NaN
2,activemq,19535,"LOG.debug(""{} received with excessive redelive...",debug,https://github.com/apache/activemq/blob/main/a...,private MessageDispatch dequeue(long timeout) ...,NaN,NaN
3,activemq,18908,"LOG.info(""no limits set, slowConsumer strategy...",info,https://github.com/apache/activemq/blob/main/a...,@Override\n public void slowConsumer(Connec...,NaN,NaN
4,activemq,19815,"LOG.trace(""Ignoring exception on close as disc...",trace,https://github.com/apache/activemq/blob/main/a...,@Override\n public void close() throws JMSE...,NaN,NaN
...,...,...,...,...,...,...,...,...
1350,jmeter,14349,"log.debug(""Writing header to file: {}"", filename)",debug,https://github.com/apache/jmeter/blob/master/s...,private static PrintWriter getFileWriter(final...,NaN,NaN
1351,jmeter,14958,"log.warn(""Cannot read init file: {}"", fileToUse)",warn,https://github.com/apache/jmeter/blob/master/s...,private void init() throws ClassNotFoundExcept...,NaN,NaN
1352,jmeter,13734,"log.error(""Error instantiating class:{}:{}"", t...",error,https://github.com/apache/jmeter/blob/master/s...,private static Object getClassInstance(String ...,NaN,NaN
1353,jmeter,14788,"log.warn(""Engine is busy - cannot create JMete...",warn,https://github.com/apache/jmeter/blob/master/s...,@Override\n public void rconfigure(HashTree...,NaN,NaN


**Note**:

All the data are sampled using a 95% confidence level with a confidence interval of 5:

Hadoop: 13204 -> 

Jmeter: 1932 ->

Tomcat: 2590 ->

ActivqMQ: 2139 ->

In [4]:
sampled_logs.loc[sampled_logs['Project'] == 'activemq'].shape

(326, 8)

In [5]:
sampled_logs.loc[sampled_logs['Project'] == 'jmeter'].shape

(321, 8)

In [6]:
sampled_logs.loc[sampled_logs['Project'] == 'hadoop'].shape

(373, 8)

In [7]:
sampled_logs.loc[sampled_logs['Project'] == 'tomcat'].shape

(335, 8)

### Get all data from database

In [11]:
import psycopg2
import json

In [12]:
with open('../config.json', 'r') as f:
    config = json.load(f)

conn = psycopg2.connect(database=config['db']['database'], user=config['db']['user'], password=config['db']['password'], host=config['db']['host'],
                        port=config['db']['port'])

In [13]:
def fetch_all_logs():
    cur = conn.cursor()
    sql = "select * from log_statements_for_log_code_inconsistency_detection where commit IS NULL order by id limit 19865;"
    cur.execute(sql)
    rows = cur.fetchall()
    cur.close()
    return rows

In [14]:
all_logs = fetch_all_logs()

In [15]:
len(all_logs), all_logs[0]

(19865,
 ('hadoop/hadoop-yarn-project/hadoop-yarn/hadoop-yarn-client/src/main/java/org/apache/hadoop/yarn/client/cli/TopCLI.java',
  'LOG.error("Caught exception", e)',
  385,
  385,
  1,
  None,
  385,
  385,
  21,
  38,
  'hadoop',
  None))

### Get all logs need to be detected

In [16]:
sampled_logs_ids = sampled_logs['log ID'].tolist()

In [17]:
logs_to_check = []
for log_record in all_logs:
    if log_record[4] in sampled_logs_ids:
        continue
    else:
        logs_to_check.append(log_record)

In [18]:
len(logs_to_check)

18510

In [19]:
log_stat_records = [x[1] for x in logs_to_check]

In [20]:
len(log_stat_records),log_stat_records[0]

(18510, 'LOG.error("Caught exception", e)')

### Log text extraction: process the raw data 

In [21]:
import pickle

with open('../data/all_local_strings.pkl', 'rb') as f:
    all_local_strings = pickle.load(f)

In [22]:
import re
from bs4 import BeautifulSoup
import subprocess


def get_xml_representation(log_message):
    with open('tmp.java', 'w') as f:
        f.write(log_message)
    _cmd_srcml = "srcml {} -o {}"
    subprocess.getoutput(_cmd_srcml.format('tmp.java', 'tmp.xml'))

    
def check_previous_sibling_operator(node):
    if node.previous_sibling is None:
        return False
    if node.previous_sibling.name is None:
        node = node.previous_sibling
    node = node.previous_sibling
    if node is not None and node.string.strip() == '+':
        return True
    else:
        return False
    
    
def check_next_sibling_operator(node):
    if node.next_sibling is None:
        return False
    if node.next_sibling.name is None:
        node = node.next_sibling
    node = node.next_sibling
    if node is not None and node.string.strip() == '+':
        if node.next_sibling.name is None:
            node = node.next_sibling
        if node.next_sibling is not None and node.next_sibling.name == 'literal':
            return False
        else:
            return True
    else:
        return False

    
def check_ternary(node):
    if node.name == 'literal':
        for x in node.parents:
            if x.name == "ternary":
                return False
        return True
    else:
        return False


def replace_placeholder(log_text):
    pattern = re.compile(r'((%[^A-Za-z]*[a-hnostx]+)|{\d*})')
    return re.sub(pattern, ' VID ', log_text)


def replace_local_strings(all_local_strings, log_stat_stripped_no_localizer):
    for string_key, string_value in all_local_strings.items():
        if '"'+string_key+'"' in log_stat_stripped_no_localizer:
            log_stat_stripped_no_localizer = log_stat_stripped_no_localizer.replace(string_key,
                                                                                    string_value)
            return log_stat_stripped_no_localizer.replace('[{', '{').replace('}]', '}')
    return log_stat_stripped_no_localizer


def check_local_str_replace(all_local_strings, log_stat):
    if "Localizer.getMessage" in log_stat or "sm.getString" in log_stat:
        log_stat = replace_local_strings(all_local_strings, log_stat)
    return log_stat

def extract_text_from_log_stat(all_local_strings, log_stat):
    log_stat = check_local_str_replace(all_local_strings, log_stat)
    final_text = []
    pattern = re.compile(r'((%[^A-Za-z]*[a-hnostx]+)|{\d*})')
    log_message = re.search("\(((.|\n)*)\)", log_stat).group(1)
    get_xml_representation(log_message)
    soup = BeautifulSoup(open('tmp.xml'), "xml")
    log_text_list = soup.find_all(check_ternary)
    for log_text_id, log_text in enumerate(log_text_list):
        if log_text_id == 0 and check_previous_sibling_operator(log_text):
            final_text.append("VID")
            final_text.append(log_text.string.replace('"', ''))
            if check_next_sibling_operator(log_text):
                final_text.append("VID")
        else:
            if re.search(pattern, log_text.string) is not None:
                final_text.append(replace_placeholder(log_text.string))
            else:
                final_text.append(log_text.string.replace('"', ''))
                if check_next_sibling_operator(log_text):
                    final_text.append("VID")
    return ' '.join(final_text).replace('"', '')

In [ ]:
logs_text = [extract_text_from_log_stat(all_local_strings, x) for x in log_stat_records]

### Detect the tensen of the log text

In [23]:
import sys
sys.path.append('../')

In [24]:
import importlib
import tempolo_semantic
from tempolo_semantic import get_log_content_rel

In [25]:
test_log_text = "Can't handle this event at current state"
get_log_content_rel(test_log_text)

'after'

In [ ]:
logs_realtion = []
for x in tqdm(logs_text):
    logs_realtion.append(get_log_content_rel(x))

### Detect inconsistency

In [27]:
import importlib
import inconsistency_detection
from inconsistency_detection import inconsistency_detection

In [28]:
from tqdm.notebook import tqdm

In [ ]:
detected_idx = []
for idx, log_record in tqdm(enumerate(logs_to_check)):
    log_id = log_record[4]
    log_content_relation = logs_realtion[idx]
    log_text = logs_text[idx]
    if log_content_relation is not None:
        try:
            res = inconsistency_detection(log_content_relation, log_id, log_text)
            if res:
                detected_idx.append(idx)
        except Exception as e:
            continue

## Analyze the manually labeled data

In [29]:
pickle_path = "../data/ground_truth.csv"
incorrect_logs = pd.read_csv(pickle_path)

In [30]:
incorrect_logs

,Project,Log ID,Log,Log level,Log URL,Log content relation,Log-code relation
0,jmeter,13682,"log.warn(""Existing CookieManager {} superseded...",warn,https://github.com/apache/jmeter/blob/master/s...,after,before
1,activemq,18247,"LOG.debug(""Session {} closed"", getSessionId())",debug,https://github.com/apache/activemq/blob/b3c2c4...,after,before
2,hadoop,13139,"LOG.info(""Local service "" + localTarget + "" en...",info,https://github.com/apache/hadoop/blob/dbd255f4...,after,meets/starts
3,jmeter,13683,"log.warn(""Existing KeystoreConfig {} supersede...",warn,https://github.com/apache/jmeter/blob/659c1ff5...,after,meets/starts
4,jmeter,13679,"log.warn(""Existing AuthManager {} superseded b...",warn,https://github.com/apache/jmeter/blob/659c1ff5...,after,meets/starts
5,jmeter,15098,"log.info(""Thread started: {}"", Thread.currentT...",info,https://github.com/apache/jmeter/blob/659c1ff5...,after,before
6,activemq,19714,"LOG.debug(""Stopped {}"", this)",debug,https://github.com/apache/activemq/blob/b3c2c4...,after,meets/starts
7,activemq,18092,"LOG.info(""Stopped KahaDB"")",info,https://github.com/apache/activemq/blob/b3c2c4...,after,meets/starts
8,activemq,17814,"LOG.debug(""Stopped recover next messages"")",debug,https://github.com/apache/activemq/blob/b3c2c4...,after,meets/starts
9,hadoop,287,"log.info(""Reading config from: "" + configFile....",info,https://github.com/apache/hadoop/blob/trunk/ha...,meets/starts,after


In [31]:
incorrect_logs_ids = list(incorrect_logs['Log ID'])
len(incorrect_logs_ids), incorrect_logs_ids[0]

(33, 13682)

In [32]:
def fetch_log_context_by_log_id(log_id):
    cur = conn.cursor()
    sql = "SELECT * FROM log_statements_for_log_code_inconsistency_detection WHERE id=%s;"
    cur.execute(sql, (log_id,))
    rows = cur.fetchall()
    cur.close()
    return rows

In [33]:
logs_to_check = []
for incorrect_logs_id in incorrect_logs_ids:
    logs_to_check += fetch_log_context_by_log_id(incorrect_logs_id)

In [34]:
len(logs_to_check), logs_to_check[0]

(33,
 ('jmeter/src/protocol/http/src/main/java/org/apache/jmeter/protocol/http/sampler/HTTPSamplerBase.java',
  'log.warn("Existing CookieManager {} superseded by {}", mgr.getName(), value.getName())',
  954,
  954,
  13682,
  None,
  954,
  954,
  26,
  69,
  'jmeter',
  None))

In [35]:
log_stat_records = [x[1] for x in logs_to_check]
logs_text = [extract_text_from_log_stat(all_local_strings, x) for x in log_stat_records]

In [36]:
logs_realtion = []
for x in tqdm(logs_text):
    logs_realtion.append(get_log_content_rel(x))

  0%|          | 0/33 [00:00<?, ?it/s]

In [37]:
# None means error detection of spaCy
len(logs_realtion), logs_realtion

(33,
 ['after',
  'after',
  'after',
  'after',
  'after',
  'after',
  'after',
  'after',
  'after',
  'meets/starts',
  'meets/starts',
  'meets/starts',
  'meets/starts',
  'meets/starts',
  'meets/starts',
  'meets/starts',
  'meets/starts',
  'meets/starts',
  'meets/starts',
  'meets/starts',
  'meets/starts',
  'meets/starts',
  'meets/starts',
  'meets/starts',
  'meets/starts',
  'meets/starts',
  'meets/starts',
  None,
  'meets/starts',
  'meets/starts',
  'after',
  'meets/starts',
  None])

In [38]:
detected_idx_1 = []
for idx, log_record in tqdm(enumerate(logs_to_check)):
    log_id = log_record[4]
    log_content_relation = logs_realtion[idx]
    log_text = logs_text[idx]
    if log_content_relation is not None:
        try:
            res = inconsistency_detection(log_content_relation, log_id, log_text)
            if res:
                detected_idx_1.append(idx)
        except Exception as e:
            continue

0it [00:00, ?it/s]

superseded
Existing CookieManager VID superseded by VID
Detected by Rule 2.
closed
Session VID closed
Detected by Rule 1.
entered
Local service VID entered state: VID
Detected by Rule 1.
superseded
Existing KeystoreConfig VID superseded by VID
Detected by Rule 2.
superseded
Existing AuthManager VID superseded by VID
Detected by Rule 2.
started
Thread started: VID
Detected by Rule 2.
Stopped
Stopped VID
Detected by Rule 13-0.
Stopped
Stopped KahaDB
Detected by Rule 2.
Stopped
Stopped recover next messages
Detected by Rule 3.
writing
writing to: VID
Detected by Rule 4.
Writing
Writing out partial crc for data len VID ,
Detected by Rule 8.
Delete
Delete resource skylines for VID .
Detected by Rule 6.
Instantiating
Instantiating AHSWebApp at VID
Detected by Rule 6.
Request
Request for appInfo of unknown attempt VID
Setting
Setting transaction isolation: VID @ VID
Detected by Rule 6.
Adding
Adding event to selector: VID
Detected by Rule 8.
Trying
Trying to get Root Broker
Detected by Rule 8

In [39]:
len(detected_idx_1)

26

In [40]:
26/33

0.7878787878787878

## Analyze the commit history data

In [41]:
log_ids = [20678, 20686, 20716, 20730, 20732, 20734]

In [43]:
def fetch_log_stat_by_log_id(log_id):
    cur = conn.cursor()
    sql = "SELECT * FROM log_statements_for_log_code_inconsistency_detection WHERE id=%s;"
    cur.execute(sql, (log_id,))
    rows = cur.fetchall()
    cur.close()
    return rows

In [44]:
logs_to_check = []
for log_id in log_ids:
    logs_to_check.append(fetch_log_stat_by_log_id(log_id)[0])

In [45]:
len(logs_to_check), logs_to_check[0]

(6,
 ('jmeter/src/core/org/apache/jmeter/testelement/TestPlan.java',
  'log.info("add " + path + " to classpath")',
  257,
  257,
  20678,
  'dd40b8279a32d6ebadc5b4b8ded06dd67935f0af',
  257,
  257,
  22,
  52,
  'jmeter',
  'commit'))

In [46]:
log_stat_records = [x[1] for x in logs_to_check]

In [47]:
len(log_stat_records),log_stat_records[0]

(6, 'log.info("add " + path + " to classpath")')

In [49]:
logs_text = [extract_text_from_log_stat(all_local_strings, x) for x in log_stat_records]

In [50]:
logs_text

['add  VID  to classpath',
 'Ending thread  VID',
 'End : sample, redirects= VID',
 '***DB is being shutdown...',
 '========== finished  VID  ==========',
 'Starting broker..']

In [51]:
logs_realtion = []
for x in tqdm(logs_text):
    logs_realtion.append(get_log_content_rel(x))

  0%|          | 0/6 [00:00<?, ?it/s]

In [53]:
detected_idx = []
for idx, log_record in tqdm(enumerate(logs_to_check)):
    log_id = log_record[4]
    log_content_relation = logs_realtion[idx]
    log_text = logs_text[idx]
    if '========== ' in log_text:
        log_text = log_text.strip('========== ').capitalize()
    if log_content_relation is not None:
        try:
            if inconsistency_detection(log_content_relation, log_id, log_text):
                detected_idx.append(idx)
        except:
            continue

0it [00:00, ?it/s]

add
add VID to classpath
Detected by Rule 4.
Ending
Ending thread VID
redirects=
redirects= VID
Detected by Rule 0-0.
shutdown
***DB is being shutdown...
Detected by Rule 8.
Finished
Finished vid
Detected by Rule 1.
Starting
Starting broker..
Detected by Rule 4.


In [54]:
len(detected_idx)

5

In [55]:
5/6

0.8333333333333334